# VIX-Adjusted Momentum (US)

Replicating [VIX-Adjusted Momentum](https://cssanalytics.wordpress.com/2014/07/29/vix-adjusted-momentum/)

In [ ]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()

In [ ]:
startDate <- as.Date('1990-01-02')
endDate <- as.Date('2019-06-30')

vixDt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^VIX' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()
    
sp500Dt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^GSPC' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

allXts <- merge(sp500Xts <- xts(sp500Dt$CLOSE, sp500Dt$TIME_STAMP), xts(vixDt$CLOSE, vixDt$TIME_STAMP))
names(allXts) <- c('SP500', 'VIX')


In [ ]:
allXts$SMA_200 <- SMA(allXts$SP500, 200)
allXts$D_RET <- dailyReturn(allXts$SP500)
allXts$D_RET_LAG_1 <- stats::lag(allXts$D_RET, -1) #signal is generated and traded at the close, so use next day's return

allXts$VIX_ADJ_RET <- allXts$D_RET/allXts$VIX
allXts$VIX_ADJ_RET_SMA_200 <- SMA(allXts$VIX_ADJ_RET, 200) 

allXts$ROC_200 <- rollapply(allXts$D_RET, 200, Return.cumulative) #simple compound return or ROC (rate of change) 

allXts <- na.omit(allXts)
print(head(allXts))
print(tail(allXts))

In [ ]:
#go long only if the index is above its 200-day average
allXts$SMA_STRAT <- ifelse(allXts$SP500 > allXts$SMA_200, allXts$D_RET_LAG_1, 0) 

#go long only if the simple average (VIX-Adjusted Momentum) > 0
allXts$VIX_STRAT <- ifelse(allXts$VIX_ADJ_RET_SMA_200 > 0, allXts$D_RET_LAG_1, 0) 

#go long only if the ROC > 0
allXts$ROC_200_STRAT <- ifelse(allXts$ROC_200 > 0, allXts$D_RET_LAG_1, 0)

In [ ]:
#plot returns
toPlot <- merge(allXts$VIX_STRAT, allXts$SMA_STRAT, allXts$ROC_200_STRAT, allXts$D_RET_LAG_1)
Common.PlotCumReturns(toPlot, "Back-test", "")

In [ ]:
#calculate annual returns
byYear <- merge(yearlyReturn(allXts$VIX_STRAT), 
                yearlyReturn(allXts$SMA_STRAT),
                yearlyReturn(allXts$ROC_200_STRAT),
                yearlyReturn(allXts$D_RET_LAG_1))

byYear <- byYear * 100.0
names(byYear) <- c('VIX-Momentum', 'SMA', 'ROC', 'BH')

print(byYear)

In [ ]:
#plot annual returns
byYearDf <- data.frame(byYear)
byYearDf$T <- year(index(byYear))

ggplot(melt(byYearDf, id='T'), aes(x=T, y=value, fill=variable)) +
    theme_economist() +
    geom_bar(stat="identity", position=position_dodge()) +
    scale_x_continuous(labels=byYearDf$T, breaks=byYearDf$T) +
    geom_text_repel(aes(label= round(value, 2)), position = position_dodge(0.9)) +
    labs(x='', y='(%)', fill='', title=sprintf("Annual Returns")) +
    annotate("text", x=max(byYearDf$T), y=as.numeric(min(byYear, na.rm=T)), 
             label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)